In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import spikeinterface as si
from dartsort.localize import localize_torch as loc

path_to_dataset = '/mnt/data/noah/IBL_data_CEED/febb430e-2d50-4f83-87a0-b5ffbb9a4943/'
path2 = '/mnt/data/noah/IBL_data_CEED/dataset/'

In [ ]:
#load recording
pathREC = r'/mnt/data/noah/IBL_data_CEED/febb430e-2d50-4f83-87a0-b5ffbb9a4943/' #session 2
rec = si.read_binary_folder(pathREC)

In [ ]:
geom = np.load(path2 + 'geom_sess2.npy')
templates = np.load(path2 + 'templates_sess2.npy')
spikes_idx = np.load(path2 + 'spike_idx_sess2.npy') #spike time, channel location of max channel, neuron id from spike sorting; for each spike
chans_idx = np.load(path2 + 'chan_idx_sess2.npy')

In [ ]:
#dirty fix for going out of range:
if spikes_idx[0][-1] != 0:
    spikes_idx[0][-1]=0
    print('change')

downsampled_spikes = spikes_idx[:,::20]

spikes_idx = downsampled_spikes

In [ ]:
spikes_idx.shape

In [ ]:
#get amplitudes vector
amplitudes = rec.get_traces()[spikes_idx[0]] #use spike times as index to grab spikes amplitudes
print(amplitudes.shape) #spikes, channels. max amplitude of each spike on each channel

In [ ]:
positions = spikes_idx[0]

spike_samples = []
for posit in positions:
    spike_samples.append(list(range(posit-43,posit+78)))
spike_samples = [item for sublist in spike_samples for item in sublist] 

In [ ]:
channel_indexs = spikes_idx[1] #need to extend each entry by 121 samples to match shape
channels_extended = []
for chan in channel_indexs:
    channels_extended.append([chan]*121)
channels_extended = [item for sublist in channels_extended for item in sublist]

In [ ]:
# channels_extended

In [ ]:
# multichan = np.asarray(channels_extended)
# multichan2 = multichan
# for i in range(-5,5+1):
#     if i == 0:
#         continue
#     multichan2 = np.vstack((multichan2, multichan+i))
# # multichan2 = np.delete(multichan2, 0, 0)

In [ ]:
multichan = np.empty((11, len(channels_extended)), dtype = 'int')
mainchan = np.asanyarray(channels_extended)
for i, offset in enumerate(range(-5,5+1)):
    multichan[i] = mainchan + offset
    # if i == 0:
    #     continue
    # multichan2 = np.vstack((multichan2, multichan+i))
# multichan2 = np.delete(multichan2, 0, 0)

In [ ]:
# for i in range(11):
#     print(multichan2[i])
# print('\n')
# for i in range(11):
#     print(multichan[i])

In [ ]:
# print(len(multichan.T))

In [ ]:
# multichan2.shape

In [ ]:
# help(rec.get_traces)

In [ ]:
waveforms = rec.get_traces()[spike_samples, multichan]
# waveforms2 = rec.get_traces()[spike_samples, multichan2]

In [ ]:
waveformz = rec.get_traces()[spike_samples, channels_extended]
waveformz.shape


In [ ]:
print(waveformz.shape)
new_x = int(waveformz.shape[0]/121)
# # reshape waveform array
waveformz = waveformz.reshape((new_x, 121))
print(waveformz.shape)

In [ ]:
print(waveforms.shape)
new_x = int(waveforms.shape[1]/121)
# # reshape waveform array
waveforms = waveforms.reshape((11, new_x, 121)) #need to start using einops lol, reshaping is hard
print(waveforms.shape)

In [ ]:
plt.plot(waveformz[0])
plt.plot(waveforms[5,0,:])
plt.show()

In [ ]:
# waveforms are now multichannel!
waveforms = np.swapaxes(waveforms, 0, 1)
plt.plot(waveforms[0,5,:])
plt.show()
print(waveforms.shape)

In [ ]:
#Denoise the waveforms with the multichannel denoiser!
#no multichannel denoiser, but can run single channel lots of times:

# from dartsort.transform import single_channel_denoiser as scd
path = r'/home/noah/code/dartsort/pretrained/single_chan_denoiser.pt'
# dns = scd.SingleChannelDenoiser()
# dns.load(path)
import torch
from spike_psvae import denoise

dn = denoise.SingleChanDenoiser().load(path)

for i, channel in enumerate(waveforms.swapaxes(0,1)):
    waveforms[:,i,:] = dn(torch.FloatTensor(channel).reshape(-1, 121)).reshape(channel.shape).detach().numpy()


In [ ]:
plt.plot(waveforms[0,5,:])
plt.show()
print(waveforms.shape)

In [ ]:
# def normalize_wf(wf):
#     """Normalize waveform for use in cell type experiments.
#     Parameters
#     ----------
#     wf: numpy.ndarray
#         spike that has shape (n_chans, spike_length_samples)
#     Returns
#     -------
#     waveform normalized to the range [0, 1] to retain shape information, but not amplitude information
#     """
#     if len(wf.shape) == 1:
#         _ = wf.shape
#         n_chans = None
#     else:
#         n_chans, _ = wf.shape
#     wf = wf.flatten()
#     if np.max(np.abs(wf), axis=0) != 0:
#         wf /= np.max(np.abs(wf), axis=0)
#     wf = wf.reshape(n_chans, -1) if n_chans is not None else wf
#     return wf

In [ ]:
radius = 100 #microns #100 correct
num_spikes_st = 0
num_spikes = -1
main_chan = spikes_idx[1]
subset = None
locs = loc.localize_amplitude_vectors(amplitude_vectors = amplitudes[0:num_spikes], n_channels_subset = subset, geom = geom, main_channels = main_chan[0:num_spikes], radius = radius)


In [ ]:
z = locs["z_abs"]
x = locs["x"]
pitch = 40

idx_good = np.flatnonzero(np.logical_and(
    np.logical_and(x > geom[:, 0].min()-pitch, x < geom[:, 0].max()+pitch),
    np.logical_and(z > geom.min()-pitch, z < geom.max()+pitch)))

x = x[idx_good]
z = z[idx_good]
index = spikes_idx[2][idx_good]
waveforms = waveforms[idx_good, :, :]
amplitudes = amplitudes[idx_good, :]

In [ ]:
import colorcet as cc
def get_ccolor(k):
    if k == -1:
        return "#808080"
    else:
        return ccolors[k % len(ccolors)]
# ccolors = cc.glasbey[:31]
ccolors = cc.glasbey[:50]

cols = [get_ccolor(idx) for idx in index]

In [ ]:
plt.figure(figsize=(2,12)) #2,12 is better tbh

plt.scatter(x, z, alpha=0.2, s = 0.5, c = cols) #coloured by neuron id, adds nothing but confirms things work!
plt.scatter(geom[:,0], geom[:,1], c='red', alpha=0.5, s=0.9)
plt.xlabel('X-position')
plt.ylabel('Z absolute')
#2x12
plt.show()

In [ ]:
from ceed.models.ceed import CEED
import numpy as np
import os
import matplotlib.pyplot as plt
from analysis.projections import learn_manifold_umap, pca_train, pca
import colorcet as cc
import torch

from tqdm import tqdm

from mpl_toolkits.mplot3d import Axes3D

In [ ]:
fc_spikesorting_ckpt_dir = '/home/noah/code/ceed_models/spikesort'

fc_spikesorting_ceed_5d = CEED(num_extra_chans=5, out_dim=5, proj_dim=5, gpu=0, old_ckpt=True)
fc_spikesorting_ceed_5d.load(fc_spikesorting_ckpt_dir)

In [ ]:
print(waveforms.shape)
batch = 1
cells = waveforms.shape[0]
timepoints = waveforms.shape[2]
channels = waveforms.shape[1]

In [ ]:
waveforms = np.asarray(waveforms.squeeze())

In [ ]:
# waveforms = torch.from_numpy(waveforms.astype('float32')) #must be float32?
waveforms = waveforms.reshape(batch, int(cells/batch), channels, timepoints)
# waveforms = waveforms.to('cuda') 

#put on the gpu in the inference loop?
#https://discuss.pytorch.org/t/dataset-location-runtimeerror-caught-runtimeerror-in-dataloader-worker-process-0/156842

# print(waveforms.shape, waveforms.device)

In [ ]:
embeddings_all = []
with torch.no_grad():
    for spike in tqdm(waveforms):
        # data = torch.unsqueeze(spike, dim=-1)
        # spike.to('cuda')
        feature = fc_spikesorting_ceed_5d.transform(spike) #returns a np array
        embeddings_all.append(feature)
# embeddings_all = torch.concatenate(embeddings_all).to('cpu').detach().squeeze().numpy()
embeddings_all = np.asarray(embeddings_all).squeeze()

In [ ]:
waveforms = waveforms.squeeze()

In [ ]:
print(embeddings_all.shape)
print(waveforms.shape)

In [ ]:
def qplot(i):
    ax[i].scatter(x, z, alpha=0.2, s = 0.5, c = embeddings_all[:,i])
    ax[i].scatter(geom[:,0], geom[:,1], c='red', alpha=0.5, s=0.9)
    ax[i].set_xlabel('X-position')
    if i == 0:
        ax[i].set_ylabel('Z absolute')
    title = 'CEED ' + str(i)
    ax[i].set_title(title)
    # ax[i].set_ylim(0,1000)
    # ax[i].axis('equal')
    # ax[i].set_xlim(-1,1)

fig, ax = plt.subplots(1,5, sharey = True)
fig.suptitle('CEED Embeddings vs Geometry', fontsize='large')

for i in range(5):
    qplot(i)

# fig.colorbar(ax[4], cax=embeddings_all[:,4])
# plt.colorbar(fig)
plt.show()

In [ ]:
import scipy.signal as signal
def align_spikes(wfs):
# input: waveforms of shape NxT
# upsample the waveform in time first for finer scale alignment
# then downsample the waveform to its original shape
    N, T = np.shape(wfs)
    upsampled_wfs = signal.resample(wfs, 10*T, axis = 1)
    aligned_wfs = np.zeros([N, 2500]) 
    peak_idx = 1050 # a value that is larger than all peak index values
    peak_point = np.argmax(np.absolute(upsampled_wfs), axis =1)    
    shift = peak_idx - peak_point

    for i in range(len(shift)):
        try:
            aligned_wfs[i, shift[i]:(shift[i] + 10*T)] = upsampled_wfs[i,:]
        except:
            # print(i)
            pass
        
    return  signal.resample(aligned_wfs[:,630:(630+1210 + 1)], T, axis = 1)

# aligned_wfs = align_spikes(waveforms)

In [ ]:
# from dartsort.transform import single_channel_denoiser as scd
path = r'/home/noah/code/dartsort/pretrained/single_chan_denoiser.pt'
# dns = scd.SingleChannelDenoiser()
# dns.load(path)

from spike_psvae import denoise

dn = denoise.SingleChanDenoiser().load(path)

# wfs_denoised = dn(torch.FloatTensor(aligned_wfs).reshape(-1, 121)).reshape(aligned_wfs.shape).detach().numpy()

In [ ]:
#find ptp amps:
ptp = np.asarray([np.ptp(wave) for wave in waveforms[:,5,:]]) #note that this is not denoised!

In [ ]:
spike_sorting_features = np.empty((embeddings_all.shape[0], embeddings_all.shape[1]+3), dtype='float32')

for i, ceed_features in enumerate(embeddings_all):
    spike_sorting_features[i] = np.append(ceed_features, [x[i], z[i], ptp[i]])
#shape == ceed1, ceed2, ceed3, ceed4, ceed5, x, z, ptp for every spike

In [ ]:
spike_sorting_features[0]

In [ ]:
#add location info:

# spike_sorting_features = list(zip(embeddings_all.tolist(), z, x))
# spike_sorting_features = list(map(list, zip(embeddings_all.tolist(), z, x)))

In [ ]:
# spike_sorting_features[1]

In [ ]:
# def flatten(l):
#     return [item for sublist in l for item in sublist if isinstance(l, list)]

In [ ]:
# spike_sorting_features = [flatten(feature) for feature in spike_sorting_features]

In [ ]:
spike_sorting_features.shape

## Cluster this data

In [ ]:
original_features = spike_sorting_features[:,5::] #x, y , ptp

In [ ]:
original_features[0]

In [ ]:
#best clustering is to use hdbscan

In [ ]:
import hdbscan

In [ ]:
# create list of features (scaled)

# scale is /stddev

# take distrubutions of ceed features -try to make gaussian

# scale everything to std dev of x

#note that the -1 label is for outliers

In [ ]:
# lets scale features:
std_dev_x = np.std(original_features[:, 0])
print(std_dev_x)
original_features = original_features/std_dev_x
original_features[0]

In [ ]:
clusterer = hdbscan.HDBSCAN()
clusterer.fit(original_features)

In [ ]:
og_labels = clusterer.labels_
print(len(np.unique(og_labels)))

In [ ]:
#outlier percentage
( list(clusterer.labels_).count(-1) / len(clusterer.labels_) ) * 100

In [ ]:
label = -1
mask = og_labels>label

In [ ]:
cols_original_features = [get_ccolor(idx) for idx in og_labels[mask]]

In [ ]:
# plot clusters:

plt.figure(figsize=(2,12)) #2,12 is better tbh

plt.scatter(x[mask], z[mask], alpha=0.2, s = 0.5, c = cols_original_features)
plt.scatter(geom[:,0], geom[:,1], c='red', alpha=0.5, s=0.9)
plt.xlabel('X-position')
plt.ylabel('Z absolute')
#2x12
plt.show()

In [ ]:
#visualize the units
unit = 7
for wave in waveforms[og_labels==unit][:,5,:]:
    plt.plot(wave, alpha=0.1, color='k')
plt.plot(np.mean(waveforms[og_labels==unit][:,5,::downsample], 0), alpha=1, color = 'c')
plt.show()
